# TASK 1 - Implementation of Sequence Alignment Algorithms
Zofia Łągiewka 313096

In [106]:
from utils import load_matrix, fill_matrices, traceback, print_and_save_results
import numpy as np

## Needleman-Wunsch - global alignment

In [107]:
def needleman_wunsch(sequence1, sequence2, n, path, GP_, output_filename='output_NW.txt', print_results=True):
    """
    Performs the Needleman-Wunsch algorithm to find n global alignments

    Parameters:
    - sequence1 (str): first DNA sequence
    - sequence2 (str): second DNA sequence
    - n (int): maximum number of alignments
    - path (str): filepath to the substitution matrix in CSV format
    - GP (int): gap penalty
    - output_filename (str): name of the output file
    """
    substitution_matrix_ = load_matrix(path)
    scoring_matrix, direction_matrix = fill_matrices(sequence1, sequence2, substitution_matrix_, GP_)
    
    alignments = []
    traceback(sequence1, sequence2, direction_matrix, scoring_matrix, len(sequence1), len(sequence2), '', '', alignments, n)
    
    print_and_save_results(output_filename, alignments, "Global", print_results)    
    return alignments

### Example use

In [108]:
needleman_wunsch('TATA', 'ATAT', n=3, path='matrix.csv', GP_=-2, output_filename='output_nw.txt')

Global alignment no. 1:
-TATA
ATAT-
Score: 11

Global alignment no. 2:
TATA-
-ATAT
Score: 11


[('-TATA', 'ATAT-', 11), ('TATA-', '-ATAT', 11)]

## Smith-Waterman - local alignment

In [109]:
def smith_waterman(sequence1, sequence2, n, path, GP_, output_filename='output_SW.txt', print_results=True):
    """
    Performs the Smith-Waterman algorithm to find n local alignments

    Parameters:
        - sequence1 (str): first DNA sequence
        - sequence2 (str): second DNA sequence
        - n (int): maximum number of alignments
        - path (str): filepath to the substitution matrix in CSV format
        - GP (int): gap penalty
        - output_filename (str): name of the output file
    """
    substitution_matrix_ = load_matrix(path)
    scoring_matrix, direction_matrix = fill_matrices(sequence1, sequence2, substitution_matrix_, GP_, global_alignment=False)
    
    alignments = []
    max_score = scoring_matrix.max()
    for (i, j) in np.argwhere(scoring_matrix == max_score):
        traceback(sequence1, sequence2, direction_matrix, scoring_matrix, i, j, '', '', alignments, n, global_alignment=False, start_position=(i, j))
    
    print_and_save_results(output_filename, alignments, "Local", print_results)
    return alignments

### Example use

In [110]:
smith_waterman('TATA', 'ATAT', n=3, path='matrix.csv', GP_=-2, output_filename='output_sw.txt')

Local alignment no. 1:
TAT
TAT
Score: 15

Local alignment no. 2:
ATA
ATA
Score: 15


[('TAT', 'TAT', 15), ('ATA', 'ATA', 15)]

# UNIT tests

In [111]:
seq1 = 'ATCG'
seq2 = 'TACG'
GP = -2
max_alignments = 3
PATH = 'matrix.csv'
substitution_matrix = load_matrix(PATH)

### Checking if the results are saved properly and fulfill theoretical requirements

In [136]:
def test_fill_matrices_NW(seq1_, seq2_, GP_, substitution_matrix_):
    """
    Tests the initialization of the Needleman-Wunsch scoring matrix

    Asserts:
        - gap penalties are correctly initialized in the scoring matrix
        - final score is non-negative to validate proper global alignment scoring
    """
    scoring_matrix, direction_matrix = fill_matrices(seq1_, seq2_, substitution_matrix_, GP_)
    assert scoring_matrix[0, 1] == GP_, "Global alignment gap penalty initialization failed"
    assert scoring_matrix[1, 0] == GP_, "Global alignment gap penalty initialization failed"
    assert scoring_matrix[-1, -1] >= 0, "Global alignment scoring matrix does not have expected positive score"
    print("Test passed")

In [137]:
def test_fill_matrices_SW(seq1_, seq2_, GP_, substitution_matrix_):
    """
    Tests the initialization of the Smith-Waterman scoring matrix

    Asserts:
        - matrix starts with zeros, which is a characteristic of local alignment
        - all scores are non-negative
    """
    scoring_matrix, direction_matrix = fill_matrices(seq1_, seq2_, substitution_matrix_, GP_, global_alignment=False)
    assert scoring_matrix[0, 0] == 0, "Local alignment matrix should start with zero"
    assert scoring_matrix.max() >= 0, "Local alignment scoring matrix should only contain non-negative scores"
    print("Test passed")

In [138]:
def test_saving_to_file_NW(seq1_, seq2_, GP_, max_alignments_, PATH_):
    """
    Tests whether global alignments are saved properly to a file
    """
    needleman_wunsch(seq1_, seq2_, n=max_alignments_, path=PATH_, GP_=GP_, output_filename='output_nw_test.txt', print_results=False)
    with open('output_nw_test.txt', 'r') as file:
        content = file.read()
    assert "Global alignment no. 1" in content, "Needleman-Wunsch output file does not contain expected results"
    print("Test passed")

In [139]:
def test_saving_to_file_SW(seq1_, seq2_, GP_, max_alignments_, PATH_):
    """
    Tests whether local alignments are saved properly to a file
    """
    smith_waterman(seq1_, seq2_, n=max_alignments_, path=PATH_, GP_=GP_, output_filename='output_sw_test.txt', print_results=False)
    with open('output_sw_test.txt', 'r') as file:
        content = file.read()
    assert "Local alignment no. 1" in content, "Smith-Waterman output file does not contain expected results"
    print("Test passed")

In [140]:
test_fill_matrices_NW(seq1, seq2, GP, substitution_matrix)
test_fill_matrices_SW(seq1, seq2, GP, substitution_matrix)
test_saving_to_file_NW(seq1, seq2, GP, substitution_matrix, max_alignments, PATH)
test_saving_to_file_SW(seq1, seq2, GP, substitution_matrix, max_alignments, PATH)

Test passed
Test passed
Test passed
Test passed


### Comparing with outside sources
Correct results were obtained from: 
http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Needleman-Wunsch 
http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Smith-Waterman

For the purpose of the below tests, constant match and mismatch penalties were used (match: 4, mismatch: -3, GP: -2) 

In [133]:
PATH = 'matrix_test.csv'
substitution_matrix = load_matrix(PATH)

#### Check scoring matrix

In [118]:
def test_correctness_of_filling_matrices(seq1_, seq2_, substitution_matrix_, GP_, correct_scoring_matrix, global_alignment=True):
    """
    Validates the accuracy of scoring matrix values for global or local alignment

    Asserts:
        - the computed scoring matrix matches the expected values element-wise
    """
    scoring_matrix, direction_matrix = fill_matrices(seq1_, seq2_, substitution_matrix_, GP_, global_alignment=global_alignment)
    for i in range(len(correct_scoring_matrix)):
        for j in range(len(correct_scoring_matrix[i])):
            assert scoring_matrix[i, j] == correct_scoring_matrix[i][j], f"Scoring matrix mismatch at ({i}, {j}): expected {correct_scoring_matrix[i][j]}, got {scoring_matrix[i, j]}"
    print("Test passed")

In [119]:
correct_scoring_matrix_NW = [[0, -2, -4, -6, -8],
                             [-2, -3, 2, 0, -2],
                             [-4, 2, 0, -1, -3],
                             [-6, 0, -1, 4, 2],
                             [-8, -2, -3, 2, 8]]
test_correctness_of_filling_matrices(seq1, seq2, substitution_matrix, GP, correct_scoring_matrix_NW)

Test passed


In [120]:
correct_scoring_matrix_SW = [[0, 0, 0, 0, 0],
                             [0, 0, 4, 2, 0],
                             [0, 4, 2, 1, 0],
                             [0, 2, 1, 6, 4],
                             [0, 0, 0, 4, 10]]
test_correctness_of_filling_matrices(seq1, seq2, substitution_matrix, GP, correct_scoring_matrix_SW, global_alignment=False)

Test passed


#### Check alignments and score

In [150]:
def test_correctness_of_alignments(seq1_, seq2_, PATH_, GP_, max_alignments_, correct_alignments_, correct_score_, global_alignment=True):
    """
    Verifies alignment results produced by Needleman-Wunsch or Smith-Waterman algorithms

    Asserts:
        - number of alignments matches the expected count (unless max_alignment criteria states otherwise)
        - each alignment and score matches expected values
    """
    if global_alignment:
        alignments = needleman_wunsch(seq1_, seq2_, max_alignments_, PATH_, GP_, print_results=False)
    else:
        alignments = smith_waterman(seq1_, seq2_, max_alignments_, PATH_, GP_, print_results=False)
        
    if len(alignments) < max_alignments_:
        assert len(alignments) == len(correct_alignments_), f"Expected {len(correct_alignments_)} alignments, but got {len(alignments)}"
    
    for i, (al1, al2, score) in enumerate(alignments):
        assert (al1, al2) == correct_alignments_[i], f"Alignment mismatch at index {i}: expected {correct_alignments_[i]}, got {(al1, al2)}"
        assert score == correct_score_, f"Expected alignment score {correct_score_}, but got {score}"

    print("Test passed")

In [151]:
correct_alignments_NW = [('-ATCG', 'TA-CG'), ('AT-CG', '-TACG')]
correct_score_NW = 8
test_correctness_of_alignments(seq1, seq2, PATH, GP, max_alignments, correct_alignments_NW, correct_score_NW)

Test passed


In [135]:
correct_alignments_SW = [('ATCG', 'A-CG'), ('T-CG', 'TACG')]
correct_score_SW = 10
test_correctness_of_alignments(seq1, seq2, PATH, GP, max_alignments, correct_alignments_SW, correct_score_SW, global_alignment=False)

Test passed
